In [1]:
%load_ext autoreload

In [ ]:
"""
Subjects articulate 36 distinct words that span the entire English language phonetic space in `silent' and `audible' manner. 


They are - 
0. Eager 
1. lift
2. eight
3. edge
4. cap
5. matted
6. tub
7. box
8. rune
9. rook
10. folder
11. block
12. fun
13. mop
14. pod
15. very
16. went
17. throat
18. this
19. tango
20. doubt
21. not
22. pretty
23. xerox
24. rodent
25. limb
26. batch
27. jeep
28. ship
29. beige
30. yes
31. echo
32. gold
33. sing
34. Uh-oh
35. hiccup

DATA is given in a numpy array of dimensions (360, 22, 7500) - (36 words each repeated 10 times, 22 channels, 7500 time samples).
Raw data was filtered using 3rd order Butterworth bandpass filter between 80 and 1000 Hertz. 
"""

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [3]:
from manifoldRnn import spdNN
from manifoldRnn import optimizers 
from manifoldRnn import trainTest
from manifoldRnn import spdRnn

In [4]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [5]:
dev = "cuda:0" 
device = torch.device(dev)

In [6]:
numberWords = 36
trialsPerWord = 10
numberTrials = numberWords * trialsPerWord
numberChannels = 22
windowLength = 7500

In [7]:
subjectNumber = 1
articulationManner = "Voiced"
subject = "Subject" + str(subjectNumber)

In [8]:
DATA = np.load("Experiment1/Words/" + articulationManner + subject + ".npy")
   
mean = np.mean(DATA, axis = -1)
std = np.std(DATA, axis = -1)
DATA = (DATA - mean[..., np.newaxis])/(std[..., np.newaxis] + 1e-5)

wordMatrices = DATA
labelsByWords = np.array([[i] * trialsPerWord for i in range(numberWords)]).reshape(numberTrials)

Indices =  {}
for i in range(numberWords):
    Indices[i] = []
for i in range(len(labelsByWords)):
    Indices[labelsByWords[i]].append(i)

In [9]:
slicedMatrices = np.zeros((numberWords * trialsPerWord, 46, numberChannels, numberChannels))
for j in range(numberWords * trialsPerWord):
    for i in range(46):
        where = i * 150 + 300
        start = where - 300
        End = where + 450
        slicedMatrices[j, i] = 1/750 * wordMatrices[j, :, start:End] @ wordMatrices[j, :, start:End].T

covariancesLabels = np.zeros((numberWords, trialsPerWord, 46, numberChannels, numberChannels))
for i in range(numberWords):
    for j in range(trialsPerWord):
        covariancesLabels[i, j] = slicedMatrices[Indices[i][j]]

trainFeatures = np.zeros((numberWords * 6, 46, numberChannels, numberChannels))
trainLabels = np.zeros((numberWords * 6))
count = 0
for i in range(numberWords):
    trainFeatures[count:count + 3] = covariancesLabels[i, :3]
    trainFeatures[count + 3:count + 6] = covariancesLabels[i, 5:8]
    trainLabels[count:count + 6] = [i] * 6
    count += 6

testFeatures = np.zeros((numberWords * 4, 46, numberChannels, numberChannels))
testLabels = np.zeros((numberWords * 4))
count = 0
for i in range(numberWords):
    testFeatures[count:count + 2] = covariancesLabels[i, 3:5]
    testFeatures[count + 2:count + 4] = covariancesLabels[i, 8:10]
    testLabels[count:count + 4] = [i] * 4
    count += 4

In [10]:
trainDataset = BaseDataset(trainFeatures, trainLabels)
testDataset = BaseDataset(testFeatures, testLabels)
trainDataloader = DataLoader(trainDataset, batch_size = 32, shuffle = True)
testDataloader = DataLoader(testDataset, batch_size = 32, shuffle = False)

In [11]:
numberEpochs = 150

model = spdRnn.spdRnnNet(numberWords).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)
lossFunction = nn.CrossEntropyLoss()
cnnOptimizer = optimizers.StiefelOptim(model.CNN.parameters(), lr = 0.05)
rnnOptimizer = optim.Adam(model.RNN.parameters(), lr = 0.001, weight_decay = 1e-3)

146628


In [12]:
maxValue = 0
for epoch in range(numberEpochs):
    trainLoss, trainAccuracy = trainTest.trainOperation(model, device, trainDataloader, cnnOptimizer, rnnOptimizer, lossFunction)
    testLoss, testAccuracy = trainTest.testOperation(model, device, testDataloader, lossFunction)
    if maxValue < testAccuracy:
        maxValue = testAccuracy
    print(f'Epoch: {epoch + 1}/{numberEpochs}, Training loss: {trainLoss:.4f}, Training accuracy: {trainAccuracy:.2f}%, Test loss: {testLoss:.4f}, Test accuracy: {testAccuracy:.2f}%')
print(maxValue)

Epoch: 1/150, Training loss: 0.1179, Training accuracy: 1.39%, Test loss: 0.1241, Test accuracy: 3.47%
Epoch: 2/150, Training loss: 0.1161, Training accuracy: 0.00%, Test loss: 0.1233, Test accuracy: 3.47%
Epoch: 3/150, Training loss: 0.1153, Training accuracy: 5.09%, Test loss: 0.1227, Test accuracy: 14.58%
Epoch: 4/150, Training loss: 0.1143, Training accuracy: 7.87%, Test loss: 0.1218, Test accuracy: 6.25%
Epoch: 5/150, Training loss: 0.1130, Training accuracy: 12.04%, Test loss: 0.1182, Test accuracy: 15.28%
Epoch: 6/150, Training loss: 0.1101, Training accuracy: 10.19%, Test loss: 0.1149, Test accuracy: 14.58%
Epoch: 7/150, Training loss: 0.1039, Training accuracy: 11.11%, Test loss: 0.1086, Test accuracy: 10.42%
Epoch: 8/150, Training loss: 0.1008, Training accuracy: 11.11%, Test loss: 0.1062, Test accuracy: 21.53%
Epoch: 9/150, Training loss: 0.0943, Training accuracy: 17.13%, Test loss: 0.1058, Test accuracy: 11.81%
Epoch: 10/150, Training loss: 0.0918, Training accuracy: 18.06

In [13]:
print(maxValue)

79.86111111111111


In [14]:
%autoreload